<a href="https://colab.research.google.com/github/Darkcoder1995/Analysis-of-child-mortaity-rates-using-Python-master/blob/main/Graph_Transformer_ETA_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-geometric

In [2]:
!pip install torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html


In [3]:
!pip install sodapy

In [4]:
!pip install openrouteservice

In [5]:
import os
import requests
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, GATConv
from torch_geometric.loader import NeighborLoader
from sodapy import Socrata
from geopy.distance import geodesic

from sklearn.neighbors import BallTree
import numpy as np
import requests
import time

import openrouteservice
from tqdm import tqdm
from openrouteservice import Client
from openrouteservice.exceptions import ApiError
import time
from sklearn.preprocessing import StandardScaler
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import softmax
import torch.optim as optim
from torch_scatter import scatter
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [6]:
# Step 1: Download Green Taxi Trip Data (Sept 2015)
client = Socrata("data.cityofnewyork.us", None)
results = client.get("gi8d-wdg5", limit=10000)
results_df = pd.DataFrame.from_records(results)

In [7]:
min_pickup_datetime = results_df['lpep_pickup_datetime'].min()
max_dropoff_datetime = results_df['lpep_dropoff_datetime'].max()

In [8]:
# URL of the station file
url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt"

# Define fixed-width column specs (0-based indexing, end is exclusive)
colspecs = [
    (0, 11),   # ID
    (12, 20),  # LATITUDE
    (21, 30),  # LONGITUDE
    (31, 37),  # ELEVATION
    (38, 40),  # STATE
    (41, 71),  # NAME
    (72, 75),  # GSN FLAG
    (76, 79),  # HCN/CRN FLAG
    (80, 85),  # WMO ID
]

# Define column names
column_names = [
    "station_id", "latitude", "longitude", "elevation", "state",
    "name", "gsn_flag", "hcn_crn_flag", "wmo_id"
]

# Read the file using read_fwf (fixed-width format)
stations_df = pd.read_fwf(url, colspecs=colspecs, names=column_names)

# Display a few rows
print(stations_df.head())

    station_id  latitude  longitude  elevation state                   name  \
0  ACW00011604   17.1167   -61.7833       10.1   NaN  ST JOHNS COOLIDGE FLD   
1  ACW00011647   17.1333   -61.7833       19.2   NaN               ST JOHNS   
2  AE000041196   25.3330    55.5170       34.0   NaN    SHARJAH INTER. AIRP   
3  AEM00041194   25.2550    55.3640       10.4   NaN             DUBAI INTL   
4  AEM00041217   24.4330    54.6510       26.8   NaN         ABU DHABI INTL   

  gsn_flag hcn_crn_flag   wmo_id  
0      NaN          NaN      NaN  
1      NaN          NaN      NaN  
2      GSN          NaN  41196.0  
3      NaN          NaN  41194.0  
4      NaN          NaN  41217.0  


In [9]:
# Ensure all coordinate columns are numeric
stations_df['latitude'] = pd.to_numeric(stations_df['latitude'], errors='coerce')
stations_df['longitude'] = pd.to_numeric(stations_df['longitude'], errors='coerce')

results_df['pickup_latitude'] = pd.to_numeric(results_df['pickup_latitude'], errors='coerce')
results_df['pickup_longitude'] = pd.to_numeric(results_df['pickup_longitude'], errors='coerce')
results_df['dropoff_latitude'] = pd.to_numeric(results_df['dropoff_latitude'], errors='coerce')
results_df['dropoff_longitude'] = pd.to_numeric(results_df['dropoff_longitude'], errors='coerce')

In [10]:
# Convert to radians for Haversine distance
weather_coords = np.radians(stations_df[['latitude', 'longitude']].values)
pickup_coords = np.radians(results_df[['pickup_latitude', 'pickup_longitude']].values)
dropoff_coords = np.radians(results_df[['dropoff_latitude', 'dropoff_longitude']].values)

# Build BallTree using weather station coordinates
tree = BallTree(weather_coords, metric='haversine')

# Find nearest station index for each pickup and dropoff
pickup_dist, pickup_idx = tree.query(pickup_coords, k=1)
dropoff_dist, dropoff_idx = tree.query(dropoff_coords, k=1)

# Map nearest station details back to results_df
results_df['pickup_station_id'] = stations_df.iloc[pickup_idx.flatten()]['station_id'].values
results_df['pickup_station_lat'] = stations_df.iloc[pickup_idx.flatten()]['latitude'].values
results_df['pickup_station_lon'] = stations_df.iloc[pickup_idx.flatten()]['longitude'].values

results_df['dropoff_station_id'] = stations_df.iloc[dropoff_idx.flatten()]['station_id'].values
results_df['dropoff_station_lat'] = stations_df.iloc[dropoff_idx.flatten()]['latitude'].values
results_df['dropoff_station_lon'] = stations_df.iloc[dropoff_idx.flatten()]['longitude'].values

In [11]:
# NOAA API setup
TOKEN = "MFymOYvQmUjLkLmwBZxzzzRxrcWnGVqj"
headers = {'token': TOKEN}
base_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"

# Prepare unique station IDs from both pickup and dropoff
unique_stations = pd.unique(
    pd.concat([
        results_df['pickup_station_id'],
        results_df['dropoff_station_id']
    ])
)

# Ensure dates are in YYYY-MM-DD format strings
start_date = pd.to_datetime(results_df['lpep_pickup_datetime'].min()).strftime("%Y-%m-%d")
end_date = pd.to_datetime(results_df['lpep_dropoff_datetime'].max()).strftime("%Y-%m-%d")

# Dictionary to hold weather data for each station-date
weather_dict = {}

for station_id in unique_stations:
    full_station_id = f"GHCND:{station_id}"

    # Set API parameters
    params = {
        'datasetid': 'GHCND',
        'stationid': full_station_id,
        'startdate': start_date,
        'enddate': end_date,
        'limit': 1000,       # NOAA limits responses; consider pagination
        'units': 'metric'
    }

    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()  # Raise error for bad responses
        data = response.json()

        # Parse response data
        for item in data.get('results', []):
            date = item['date'][:10]
            key = (station_id, date)  # Tuple key: (station_id, date)
            if key not in weather_dict:
                weather_dict[key] = {}
            weather_dict[key][item['datatype']] = item['value']

        time.sleep(1)  # Be polite to NOAA API (1-second delay)

    except Exception as e:
        print(f"Error fetching data for station {station_id}: {e}")

Error fetching data for station USW00094789: 503 Server Error: Service Unavailable for url: https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND%3AUSW00094789&startdate=2015-01-01&enddate=2015-12-31&limit=1000&units=metric
Error fetching data for station USC00305618: 503 Server Error: Service Unavailable for url: https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND%3AUSC00305618&startdate=2015-01-01&enddate=2015-12-31&limit=1000&units=metric


In [12]:
weather_df = pd.DataFrame([
    {'station_id': sid, 'date': date, **metrics}
    for (sid, date), metrics in weather_dict.items()
])

In [13]:
for column in weather_df.columns:
    if weather_df[column].isnull().any():
        median_value = weather_df[column].median()
        weather_df[column] = weather_df[column].fillna(median_value)

In [14]:
# 1. Ensure date format
results_df['pickup_date'] = pd.to_datetime(results_df['lpep_pickup_datetime']).dt.date
results_df['dropoff_date'] = pd.to_datetime(results_df['lpep_dropoff_datetime']).dt.date
weather_df['date'] = pd.to_datetime(weather_df['date']).dt.date

# 2. Rename weather columns BEFORE merging (avoid suffix conflicts)
pickup_weather_df = weather_df.rename(columns=lambda col: f"pickup_{col}" if col not in ['station_id', 'date'] else col)
dropoff_weather_df = weather_df.rename(columns=lambda col: f"dropoff_{col}" if col not in ['station_id', 'date'] else col)

# 3. Merge pickup weather (INNER)
pickup_matched = results_df.merge(
    pickup_weather_df,
    left_on=['pickup_station_id', 'pickup_date'],
    right_on=['station_id', 'date'],
    how='inner'
).drop(columns=['station_id', 'date'])

# 4. Pickup unmatched
pickup_unmatched = results_df.merge(
    pickup_weather_df[['station_id', 'date']],
    left_on=['pickup_station_id', 'pickup_date'],
    right_on=['station_id', 'date'],
    how='left',
    indicator=True
).query("_merge == 'left_only'").drop(columns=['station_id', 'date', '_merge'])

# Add missing pickup weather as 0
for col in pickup_weather_df.columns:
    if col not in ['station_id', 'date']:
        pickup_unmatched[col] = 0

# 5. Combine both pickup matched and unmatched
results_df_pickup = pd.concat([pickup_matched, pickup_unmatched], ignore_index=True)

# 6. Merge dropoff weather (INNER)
dropoff_matched = results_df_pickup.merge(
    dropoff_weather_df,
    left_on=['dropoff_station_id', 'dropoff_date'],
    right_on=['station_id', 'date'],
    how='inner'
).drop(columns=['station_id', 'date'])

# 7. Dropoff unmatched
dropoff_unmatched = results_df_pickup.merge(
    dropoff_weather_df[['station_id', 'date']],
    left_on=['dropoff_station_id', 'dropoff_date'],
    right_on=['station_id', 'date'],
    how='left',
    indicator=True
).query("_merge == 'left_only'").drop(columns=['station_id', 'date', '_merge'])

# Add missing dropoff weather as 0
for col in dropoff_weather_df.columns:
    if col not in ['station_id', 'date']:
        dropoff_unmatched[col] = 0

# 8. Combine matched + unmatched to form final dataset
results_df_final = pd.concat([dropoff_matched, dropoff_unmatched], ignore_index=True)

# 9. Final cleanup (in case 'station_id' and 'date' columns are present)
results_df_final.drop(columns=['station_id', 'date'], inplace=True, errors='ignore')

In [15]:
# Check for NaN values in each column of results_df
nan_counts = results_df_final.isnull().sum()

# Print the count of NaN values for each column
if nan_counts.any():
    print("Columns with NaN values:")
    print(nan_counts)
else:
    print("No columns with NaN values.")

No columns with NaN values.


In [16]:
api_key = '5b3ce3597851110001cf6248e5517d57f84a48f79ade0b581ebe78cf'

In [17]:
# prompt: read results_df_final csv file

results_df_final = pd.read_csv('processed_taxi_data.csv')

In [18]:
pd.set_option('display.max_columns', None)

display(results_df_final)

,vendorid,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,ratecodeid,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,pickup_station_id,pickup_station_lat,pickup_station_lon,dropoff_station_id,dropoff_station_lat,dropoff_station_lon,pickup_date,dropoff_date,pickup_PRCP,pickup_SNOW,pickup_SNWD,pickup_WESF,pickup_WESD,pickup_DAPR,pickup_MDPR,pickup_ADPT,pickup_ASLP,pickup_ASTP,pickup_AWBT,pickup_AWND,pickup_RHAV,pickup_RHMN,pickup_RHMX,pickup_TMAX,pickup_TMIN,pickup_WDF2,pickup_WDF5,pickup_WSF2,pickup_WSF5,pickup_WT01,pickup_WT08,pickup_WT04,pickup_WT02,pickup_WT06,pickup_TAVG,pickup_WT09,pickup_WT03,dropoff_PRCP,dropoff_SNOW,dropoff_SNWD,dropoff_WESF,dropoff_WESD,dropoff_DAPR,dropoff_MDPR,dropoff_ADPT,dropoff_ASLP,dropoff_ASTP,dropoff_AWBT,dropoff_AWND,dropoff_RHAV,dropoff_RHMN,dropoff_RHMX,dropoff_TMAX,dropoff_TMIN,dropoff_WDF2,dropoff_WDF5,dropoff_WSF2,dropoff_WSF5,dropoff_WT01,dropoff_WT08,dropoff_WT04,dropoff_WT02,dropoff_WT06,dropoff_TAVG,dropoff_WT09,dropoff_WT03,ors_distance_m,ors_duration_s
0,1,2015-02-13T19:49:20.000,2015-02-13T19:53:49.000,N,1,-73.994896,40.684578,-74.003479,40.680717,2,0.70,5.0,1.0,0.5,0.00,0.0,0.3,6.80,2,1,US1NYKN0025,40.6846,-73.9867,US1NYKN0025,40.6846,-73.9867,2015-02-13,2015-02-13,0.3,0.0,80.0,0.0,0.0,5.0,17.65,-89.0,10207.0,10179.0,-39.0,4.4,55.5,38.0,76.0,2.8,-6.0,280.0,290.0,9.4,12.5,1.0,1.0,1.0,1.0,1.0,0.4,1.0,1.0,0.3,0.0,80.0,0.0,0.0,5.0,17.65,-89.0,10207.0,10179.0,-39.0,4.4,55.5,38.0,76.0,2.8,-6.0,280.0,290.0,9.4,12.5,1.0,1.0,1.0,1.0,1.0,0.4,1.0,1.0,1189.78,190.70
1,2,2015-02-26T19:17:29.000,2015-02-26T19:22:41.000,N,1,-73.993149,40.692768,-74.002129,40.684917,1,0.93,5.0,1.0,0.5,1.36,0.0,0.3,8.16,1,1,US1NYKN0025,40.6846,-73.9867,US1NYKN0025,40.6846,-73.9867,2015-02-26,2015-02-26,0.0,0.0,127.0,0.0,0.0,5.0,17.65,-89.0,10207.0,10179.0,-39.0,4.4,55.5,38.0,76.0,2.8,-6.0,280.0,290.0,9.4,12.5,1.0,1.0,1.0,1.0,1.0,0.4,1.0,1.0,0.0,0.0,127.0,0.0,0.0,5.0,17.65,-89.0,10207.0,10179.0,-39.0,4.4,55.5,38.0,76.0,2.8,-6.0,280.0,290.0,9.4,12.5,1.0,1.0,1.0,1.0,1.0,0.4,1.0,1.0,1729.93,298.64
2,2,2015-02-17T14:10:19.000,2015-02-17T14:20:44.000,N,1,-73.991631,40.685059,-73.996834,40.680424,1,0.98,8.0,0.0,0.5,0.00,0.0,0.3,8.80,2,1,US1NYKN0025,40.6846,-73.9867,US1NYKN0025,40.6846,-73.9867,2015-02-17,2015-02-17,3.0,76.0,229.0,3.0,0.0,5.0,17.65,-89.0,10207.0,10179.0,-39.0,4.4,55.5,38.0,76.0,2.8,-6.0,280.0,290.0,9.4,12.5,1.0,1.0,1.0,1.0,1.0,0.4,1.0,1.0,3.0,76.0,229.0,3.0,0.0,5.0,17.65,-89.0,10207.0,10179.0,-39.0,4.4,55.5,38.0,76.0,2.8,-6.0,280.0,290.0,9.4,12.5,1.0,1.0,1.0,1.0,1.0,0.4,1.0,1.0,1009.86,104.94
3,2,2015-02-06T21:07:34.000,2015-02-06T21:13:26.000,N,1,-73.910004,40.775623,-73.890762,40.768871,1,1.32,6.5,0.5,0.5,0.00,0.0,0.3,7.80,2,1,USW00014732,40.7794,-73.8803,USW00014732,40.7794,-73.8803,2015-02-06,2015-02-06,0.0,0.0,150.0,0.0,0.0,5.0,17.65,-89.0,10207.0,10179.0,-39.0,5.5,55.5,38.0,76.0,-2.7,-11.0,260.0,260.0,10.3,13.9,1.0,1.0,1.0,1.0,1.0,-7.7,1.0,1.0,0.0,0.0,150.0,0.0,0.0,5.0,17.65,-89.0,10207.0,10179.0,-39.0,5.5,55.5,38.0,76.0,-2.7,-11.0,260.0,260.0,10.3,13.9,1.0,1.0,1.0,1.0,1.0,-7.7,1.0,1.0,2146.68,211.62
4,1,2015-02-19T08:41:04.000,2015-02-19T08:49:17.000,N,1,-74.000633,40.682995,-73.989708,40.702469,1,1.90,8.5,0.0,0.5,1.85,0.0,0.3,11.15,1,1,US1NYKN0025,40.6846,-73.9867,US1NYKN0025,40.6846,-73.9867,2015-02-19,2015-02-19,0.0,0.0,203.0,0.0,0.0,5.0,17.65,-89.0,10207.0,10179.0,-39.0,4.4,55.5,38.0,76.0,2.8,-6.0,280.0,290.0,9.4,12.5,1.0,1.0,1.0,1.0,1.0,0.4,1.0,1.0,0.0,0.0,203.0,0.0,0.0,5.0,17.65,-89.0,10207.0,10179.0,-39.0,4.4,55.5,38.0,76.0,2.8,-6.0,280.0,290.0,9.4,12.5,1.0,1.0,1.0,1.0,1.0,0.4,1.0,1.0,3206.94,401.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [19]:
# Define weather columns (pickup + dropoff)
weather_columns = [
    'pickup_PRCP', 'pickup_SNOW', 'pickup_SNWD', 'pickup_WESF', 'pickup_WESD', 'pickup_DAPR',
    'pickup_MDPR', 'pickup_ADPT', 'pickup_ASLP', 'pickup_ASTP', 'pickup_AWBT', 'pickup_AWND',
    'pickup_RHAV', 'pickup_RHMN', 'pickup_RHMX', 'pickup_TMAX', 'pickup_TMIN', 'pickup_WDF2',
    'pickup_WDF5', 'pickup_WSF2', 'pickup_WSF5', 'pickup_WT01', 'pickup_WT08', 'pickup_WT04',
    'pickup_WT02', 'pickup_WT06', 'pickup_TAVG', 'pickup_WT09', 'pickup_WT03',
    'dropoff_PRCP', 'dropoff_SNOW', 'dropoff_SNWD', 'dropoff_WESF', 'dropoff_WESD', 'dropoff_DAPR',
    'dropoff_MDPR', 'dropoff_ADPT', 'dropoff_ASLP', 'dropoff_ASTP', 'dropoff_AWBT', 'dropoff_AWND',
    'dropoff_RHAV', 'dropoff_RHMN', 'dropoff_RHMX', 'dropoff_TMAX', 'dropoff_TMIN', 'dropoff_WDF2',
    'dropoff_WDF5', 'dropoff_WSF2', 'dropoff_WSF5', 'dropoff_WT01', 'dropoff_WT08', 'dropoff_WT04',
    'dropoff_WT02', 'dropoff_WT06', 'dropoff_TAVG', 'dropoff_WT09', 'dropoff_WT03'
]

In [20]:
# Combine trip-related and weather columns
scale_columns = [
    'trip_distance', 'passenger_count', 'fare_amount', 'tip_amount',
    'total_amount', 'ors_distance_m', 'ors_duration_s'
] + weather_columns  # Include weather data

# Initialize scaler
scaler = StandardScaler()

# Scale the selected features
scaled_features = scaler.fit_transform(results_df_final[scale_columns])

# Replace the original values with the scaled ones
results_df_final[scale_columns] = scaled_features

In [21]:
# Make sure datetime columns are parsed correctly
results_df_final['lpep_pickup_datetime'] = pd.to_datetime(results_df_final['lpep_pickup_datetime'])
results_df_final['lpep_dropoff_datetime'] = pd.to_datetime(results_df_final['lpep_dropoff_datetime'])


# Calculate trip duration
results_df_final['trip_duration'] = (
    results_df_final['lpep_dropoff_datetime'] - results_df_final['lpep_pickup_datetime']
).dt.total_seconds() / 60

# Define weather columns
weather_columns = [col for col in results_df_final.columns if col.startswith('pickup_') or col.startswith('dropoff_')]

features = []
targets = []

for i, row in results_df_final.iterrows():
    try:
        # Ensure core numerical columns are treated as such
        # Coerce errors will turn problematic values into NaN
        #pickup_latitude = pd.to_numeric(row['pickup_latitude'], errors='coerce')
        #pickup_longitude = pd.to_numeric(row['pickup_longitude'], errors='coerce')
        #dropoff_latitude = pd.to_numeric(row['dropoff_latitude'], errors='coerce')
        #dropoff_longitude = pd.to_numeric(row['dropoff_longitude'], errors='coerce')
        trip_distance = pd.to_numeric(row['trip_distance'], errors='coerce')
        passenger_count = pd.to_numeric(row['passenger_count'], errors='coerce')
        fare_amount = pd.to_numeric(row['fare_amount'], errors='coerce')
        tip_amount = pd.to_numeric(row['tip_amount'], errors='coerce')
        total_amount = pd.to_numeric(row['total_amount'], errors='coerce')
        ors_distance_m = pd.to_numeric(row['ors_distance_m'], errors='coerce')
        ors_duration_s = pd.to_numeric(row['ors_duration_s'], errors='coerce')
        # Datetime components should be integers, but add a check
        #pickup_hour = pd.to_numeric(row['lpep_pickup_datetime'].hour, errors='coerce')
        #pickup_weekday = pd.to_numeric(row['lpep_pickup_datetime'].weekday(), errors='coerce')


        base_features = [
            #pickup_latitude,
            #pickup_longitude,
            #dropoff_latitude,
            #dropoff_longitude,
            trip_distance,
            passenger_count,
            fare_amount,
            tip_amount,
            total_amount,
            #pickup_hour,
            #pickup_weekday,
            ors_distance_m,
            ors_duration_s
        ]

        # Ensure weather features are numeric, handle NaNs if coercion happened
        weather_features = [
            pd.to_numeric(row[col], errors='coerce') for col in weather_columns
        ]

        # Replace any remaining NaNs introduced by coercion with 0 or a suitable value
        feature_vector = [0 if pd.isna(f) else f for f in base_features + weather_features]

        features.append(feature_vector)
        targets.append(row['trip_duration'])

    except Exception as e:
        print(f"❌ Skipping row {i} due to error: {e}")
        # Optionally print the row causing the error for debugging
        # print(row)


# Convert features list to a numpy array first to catch non-numeric values before PyTorch
# This can help debug if issues persist
try:
    features_np = np.array(features, dtype=np.float32)
    X = torch.tensor(features_np, dtype=torch.float)
    y = torch.tensor(targets, dtype=torch.float)

    print("✅ Feature tensor shape:", X.shape)
    print("✅ Target tensor shape:", y.shape)

except ValueError as e:
    print(f"❌ Error converting features to numpy array: {e}")
    print("Please check the 'features' list for non-numeric values.")

✅ Feature tensor shape: torch.Size([10000, 77])
✅ Target tensor shape: torch.Size([10000])


In [22]:
# Ensure `X` is defined properly before edge construction
X_tensor = torch.tensor(np.array(features, dtype=np.float32), dtype=torch.float)

# Build k-NN graph edges (k=10)
k = 10
nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(X_tensor)
distances, neighbors = nbrs.kneighbors(X_tensor)

edge_index_list = []
for i in range(X_tensor.shape[0]):
    for j in neighbors[i, 1:]:  # Skip self-edge
        edge_index_list.append([i, j])

# Convert to PyTorch tensor of shape `[2, num_edges]`
edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()

# Create `edge_attr` (optional) if needed
edge_attr = torch.tensor(distances[:, 1:], dtype=torch.float).flatten()

print(f"✅ Edge Index Shape: {edge_index.shape}")  # Should be `[2, num_edges]`
print(f"✅ Edge Attr Shape: {edge_attr.shape}")  # Should match number of edges

✅ Edge Index Shape: torch.Size([2, 100000])
✅ Edge Attr Shape: torch.Size([100000])


In [23]:
# Ensure `X`, `edge_index`, `edge_attr`, and labels `y` exist
y_tensor = torch.tensor(targets, dtype=torch.float)
y_tensor = torch.log1p(y_tensor)

# Ensure edge_index is correctly formatted
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)

# Ensure edge attributes exist
edge_attr_tensor = torch.tensor(edge_attr, dtype=torch.float)

# ✅ Create PyTorch Geometric Data object (fixes the issue)
data = Data(
    x=X_tensor,  # Node features
    edge_index=edge_index_tensor,  # Connectivity between nodes
    edge_attr=edge_attr_tensor,  # Edge attributes (distances)
    y=y_tensor  # Target trip duration values
)

print(f"✅ Data object created! Node shape: {data.x.shape}, Edge shape: {data.edge_index.shape}")

✅ Data object created! Node shape: torch.Size([10000, 77]), Edge shape: torch.Size([2, 100000])


<ipython-input-23-adfac7b809ef>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)
<ipython-input-23-adfac7b809ef>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr_tensor = torch.tensor(edge_attr, dtype=torch.float)


In [24]:
from sklearn.model_selection import train_test_split

# Get total node count
num_nodes = data.x.shape[0]

# Generate indices and split into train/test (80/20)
indices = np.arange(num_nodes)
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

# Define boolean masks for PyTorch Geometric
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_indices] = True
test_mask[test_indices] = True

# Assign masks to `Data` object
data.train_mask = train_mask
data.test_mask = test_mask

print(f"✅ Train/Test masks applied! Train nodes: {train_mask.sum().item()}, Test nodes: {test_mask.sum().item()}")

✅ Train/Test masks applied! Train nodes: 8000, Test nodes: 2000


In [25]:
# import torch
# from torch import nn
# from torch_geometric.nn import MessagePassing
# from torch_geometric.data import Data

# class GraphTransformerLayer(MessagePassing):
#     def __init__(self, dim, heads=4, dropout=0.3):
#         super().__init__(aggr='add')
#         self.heads = heads
#         self.dim = dim
#         self.scale = (dim // heads) ** -0.5

#         self.q_proj = nn.Linear(dim, dim)
#         self.k_proj = nn.Linear(dim, dim)
#         self.v_proj = nn.Linear(dim, dim)
#         self.edge_proj = nn.Linear(1, dim)  # ✅ Edge embedding

#         self.out_proj = nn.Linear(dim, dim)
#         self.dropout = nn.Dropout(dropout)
#         self.layernorm = nn.LayerNorm(dim)

#     def forward(self, data):
#         x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
#         residual = x
#         x = self.layernorm(x)

#         # ✅ Multi-Head Attention projections
#         q = self.q_proj(x).view(x.size(0), self.heads, self.dim // self.heads)
#         k = self.k_proj(x).view(x.size(0), self.heads, self.dim // self.heads)
#         v = self.v_proj(x).view(x.size(0), self.heads, self.dim // self.heads)

#         # ✅ Ensure edge embeddings align properly BEFORE propagation
#         edge_emb = self.edge_proj(edge_attr).view(edge_attr.size(0), self.heads, self.dim // self.heads)

#         # ✅ Debug Step to Confirm Matching Dimensions
#         assert edge_emb.shape[-1] == q.shape[-1], f"Shape mismatch: edge_emb {edge_emb.shape}, q {q.shape}"

#         out = self.propagate(edge_index, q=q, k=k, v=v, edge_emb=edge_emb)

#         return self.out_proj(out.view(x.size(0), self.dim)) + residual  # ✅ Residual connection

In [26]:
class GraphTransformerLayer(MessagePassing):
    def __init__(self, hidden_dim, heads=4, dropout=0.1):
        super().__init__(aggr='add')  # or 'mean'
        self.heads = heads
        self.hidden_dim = hidden_dim
        self.head_dim = hidden_dim // heads
        assert hidden_dim % heads == 0

        self.scale = self.head_dim ** -0.5

        self.q_lin = nn.Linear(hidden_dim, hidden_dim)
        self.k_lin = nn.Linear(hidden_dim, hidden_dim)
        self.v_lin = nn.Linear(hidden_dim, hidden_dim)
        self.edge_lin = nn.Linear(1, hidden_dim)  # edge_attr is 1D per edge

        self.out_lin = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_dim)

    def forward(self, x, edge_index, edge_attr):
        residual = x
        x = self.layer_norm(x)

        Q = self.q_lin(x)
        K = self.k_lin(x)
        V = self.v_lin(x)

        edge_emb = self.edge_lin(edge_attr.unsqueeze(-1))  # shape: [E, hidden_dim]

        out = self.propagate(edge_index, Q=Q, K=K, V=V, edge_emb=edge_emb, size=(x.size(0), x.size(0)))
        out = self.dropout(self.out_lin(out))

        return out + residual

    def message(self, Q_i, K_j, V_j, edge_emb, index):
        # Inside message — now safe to reshape
        Q_i = Q_i.view(-1, self.heads, self.head_dim)
        K_j = K_j.view(-1, self.heads, self.head_dim)
        V_j = V_j.view(-1, self.heads, self.head_dim)
        edge_emb = edge_emb.view(-1, self.heads, self.head_dim)

        attn_score = (Q_i * (K_j + edge_emb)).sum(dim=-1) * self.scale
        attn_score = softmax(attn_score, index)

        attn_score = self.dropout(attn_score)

        return V_j * attn_score.unsqueeze(-1)

    def aggregate(self, inputs, index, ptr=None, dim_size=None):
        out = scatter(inputs, index, dim=0, dim_size=dim_size, reduce='sum')
        return out.view(-1, self.hidden_dim)

In [27]:
class GraphTransformerNet(nn.Module):
  def __init__(self, in_dim, hidden_dim=128, num_layers=2, heads=4):
    super().__init__()
    self.input_proj = nn.Linear(in_dim, hidden_dim)
    self.layers = nn.ModuleList([
    GraphTransformerLayer(hidden_dim=hidden_dim, heads=heads) for _ in range(num_layers)
    ])
    self.norm = nn.LayerNorm(hidden_dim) # This line defines 'self.norm'
    self.output_layer = nn.Linear(hidden_dim, 1)

  def forward(self, data):
    x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
    x = self.input_proj(x)
    for layer in self.layers:
        x = layer(x, edge_index, edge_attr)
    x = self.norm(x)  # ✅ corrected this line
    return self.output_layer(x).squeeze(-1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphTransformerNet(in_dim=data.x.shape[1], hidden_dim=512, num_layers=6, heads=4).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
loss_fn = nn.MSELoss()

data = data.to(device)

for epoch in range(1, 101):
    model.train()
    optimizer.zero_grad()

    out = model(data)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
    optimizer.step()

    # Eval
    model.eval()
    with torch.no_grad():
        val_pred = model(data)
        val_loss = loss_fn(val_pred[data.test_mask], data.y[data.test_mask])

    print(f"Epoch {epoch:03d} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

Epoch 001 | Train Loss: 5.7461 | Val Loss: 4.2039
Epoch 002 | Train Loss: 4.1451 | Val Loss: 2.8832
Epoch 003 | Train Loss: 2.8371 | Val Loss: 1.8530
Epoch 004 | Train Loss: 1.8165 | Val Loss: 1.1139
Epoch 005 | Train Loss: 1.0880 | Val Loss: 0.6712
Epoch 006 | Train Loss: 0.6543 | Val Loss: 0.5354
Epoch 007 | Train Loss: 0.5265 | Val Loss: 0.6027
Epoch 008 | Train Loss: 0.5949 | Val Loss: 0.6668
Epoch 009 | Train Loss: 0.6602 | Val Loss: 0.6738
Epoch 010 | Train Loss: 0.6677 | Val Loss: 0.6325
Epoch 011 | Train Loss: 0.6257 | Val Loss: 0.5740
Epoch 012 | Train Loss: 0.5677 | Val Loss: 0.5325
Epoch 013 | Train Loss: 0.5224 | Val Loss: 0.5407
Epoch 014 | Train Loss: 0.5270 | Val Loss: 0.5794
Epoch 015 | Train Loss: 0.5611 | Val Loss: 0.6083
Epoch 016 | Train Loss: 0.5892 | Val Loss: 0.6134
Epoch 017 | Train Loss: 0.5957 | Val Loss: 0.5958
Epoch 018 | Train Loss: 0.5771 | Val Loss: 0.5649
Epoch 019 | Train Loss: 0.5470 | Val Loss: 0.5345
Epoch 020 | Train Loss: 0.5182 | Val Loss: 0.5209


In [ ]:
# Put model in eval mode
model.eval()

# Predict on the whole data
with torch.no_grad():
    y_true_log = data.y[data.test_mask].cpu()
    y_pred_log = model(data)[data.test_mask].cpu()

    # Invert log1p transform for both true and predicted
    y_true = torch.expm1(y_true_log).numpy()
    y_pred = torch.expm1(y_pred_log).numpy()

# Compute metrics on original scale
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)

print(f"✅ MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

In [ ]:
print(f"y: min={data.y.min().item():.2f}, max={data.y.max().item():.2f}, mean={data.y.mean().item():.2f}")

In [ ]:
import matplotlib.pyplot as plt

y_true = data.y[data.test_mask].cpu().numpy()
y_pred = model(data).detach().cpu().numpy()[data.test_mask.cpu().numpy()]

plt.figure(figsize=(10, 4))
plt.scatter(y_true, y_pred, alpha=0.4)
plt.xlabel("True y")
plt.ylabel("Predicted y")
plt.title("True vs Predicted y")
plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], 'r--', label='Ideal')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# from torch_geometric.data import Data

# # Ensure edge indices and attributes are also tensors
# edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)
# edge_attr_tensor = torch.tensor(data.edge_attr, dtype=torch.float)

# # Create PyTorch Geometric Data object
# data = Data(x=X, edge_index=edge_index_tensor, edge_attr=edge_attr_tensor, y=y)

In [ ]:
# from sklearn.model_selection import train_test_split

# # Get node indices
# num_nodes = data.x.shape[0]
# indices = np.arange(num_nodes)

# # Split into train/test (80/20)
# train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

# # Define masks for PyTorch Geometric
# train_mask = torch.zeros(num_nodes, dtype=torch.bool)
# test_mask = torch.zeros(num_nodes, dtype=torch.bool)

# train_mask[train_indices] = True
# test_mask[test_indices] = True

# # Assign masks to graph data object
# data.train_mask = train_mask
# data.test_mask = test_mask